In [1]:
print(Pkg.installed("IterativeSolvers"))
import IterativeSolvers

0.3.1

In [14]:
function CG(A, b, tolerance::Float64, x=nothing, M=nothing)
    
    function dot(a::Vector{Float64}, b::Vector{Float64})
        sum::Float64 = 0
        
        @simd for j = 1:length(a)
            @fastmath @inbounds sum += a[j]*b[j]
        end
        sum
    end
    n = length(b)
    
    if x == nothing
        x = ones(n)
    end
    
    if !(size(A, 1) == size(A, 2) == n == length(x))
        error("Size mis-match")
    end
        
    preconditioned = (M != nothing)
    
    bNorm::Float64 = norm(b)
    
    r::Vector{Float64} = b-A*x
    
    z::Vector{Float64} = r
    if preconditioned
        z = M\r
    end
    p::Vector{Float64} = r
    
    ρ::Float64 = dot(r, z)
    
    i::Int64 = 0
    while norm(r)/bNorm > tolerance
        Ap::Vector{Float64} = A*p #A_mul_B!(Ap, A, p)
        α::Float64 = ρ/dot(Ap, p)
        @. begin 
            x += α*p
            r -= α*Ap
        end
        if preconditioned
            z = M \ r
        else
            z = r
        end
        ρ₀::Float64 = ρ
        ρ = dot(r, z)
        #β::Float64 = ρ/ρ₀
        p = z + (ρ/ρ₀)*p
        
        i += 1
    end
    x
end

CG (generic function with 3 methods)

In [29]:
n = 1000_000
#generate a symmetric positive definate matrix to test with
A = sprand(n, n, 50/n)
A = .5*(A + A') + n*.5*speye(n)

x = randn(n)
normX = norm(x)
b = A*x

gc()

#println("base tolerance: ", norm(A\b-x)/normX)
println("Iter Solve tol: ", norm(IterativeSolvers.cg(A, b, tol=1e-16)-x)/normX)
gc()
println("CG's tolerance: ", norm(CG(A, b, 1e-16)-x)/normX)

gc()

#@time A\b
#@time IterativeSolvers.cg(A, b, tol=1e-16)
#gc()
#@time CG(A, b, 1e-16)

loopCount = 15
tic()
for i = 1:loopCount
    IterativeSolvers.cg(A, b, tol=1e-16)
end
toc()

gc()

tic()
for i = 1:loopCount
    CG(A, b, 1e-16)
end
toc()

0

Iter Solve tol: 4.929620603655484e-16
CG's tolerance: 6.564946556109559e-16
elapsed time: 78.340530308 seconds
elapsed time: 71.748664088 seconds


0

In [26]:
@code_warntype CG(A, b, 1e-6)

Variables:
  #self#::#CG
  A::SparseMatrixCSC{Float64,Int64}
  b::Array{Float64,1}
  tolerance::Float64

Body:
  begin 
      return $(Expr(:invoke, MethodInstance for CG(::SparseMatrixCSC{Float64,Int64}, ::Array{Float64,1}, ::Float64, ::Void, ::Void), :(#self#), :(A), :(b), :(tolerance), :(Main.nothing), :(Main.nothing)))
  end::Array{Float64,1}
